# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
J. Shi  ->  J. Shi  |  ['J. Shi']


A. Müller  ->  A. Müller  |  ['A. Müller']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
C. Gapp  ->  C. Gapp  |  ['C. Gapp']
Arxiv has 82 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.00435


extracting tarball to tmp_2506.00435...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.00669


extracting tarball to tmp_2506.00669...

...

extracting tarball to tmp_2506.01632...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.01733


extracting tarball to tmp_2506.01733...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.01771


/tmp/ipykernel_3627/2822249172.py:52: LatexWarning: 2506.01771 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.00669-b31b1b.svg)](https://arxiv.org/abs/2506.00669) | **On Linking Planet Formation Models, Protoplanetary Disk Properties, and Mature Gas Giant Exoplanet Atmospheres**  |
|| A. D. Feinstein, et al. -- incl., <mark>K. Schwarz</mark>, <mark>E.-M. Ahrer</mark>, <mark>I. J. M. Crossfield</mark>, <mark>L. Kreidberg</mark>, <mark>G. Perotti</mark> |
|*Appeared on*| *2025-06-03*|
|*Comments*| *24 pages (37 pages with references), 8 figures, 1 table. Submitted to AAS Journals. This article is intended to reflect the discussions and perspectives of workshop participants, and not provide a comprehensive review of the fields covered*|
|**Abstract**|            Measuring a single elemental ratio (e.g., carbon-to-oxygen) provides insufficient information for understanding the formation mechanisms and evolution that affect our observations of gas giant planet atmospheres. Although the fields of planet formation, protoplanetary disks, and exoplanets are well established and interconnected, our understanding of how to self-consistently and accurately link the theoretical and observational aspects of these fields together is lacking. To foster interdisciplinary conversations, the Max-Planck Institut für Astronomie (MPIA) hosted a week-long workshop called, "Challenge Accepted: Linking Planet Formation with Present-Day Atmospheres." Here, we summarize the latest theories and results in planet formation modeling, protoplanetary disk observations, and atmospheric observations of gas giant atmospheres to address one of the challenges of hosting interdisciplinary conferences: ensuring everyone is aware of the state-of-the-art results and technical language from each discipline represented. Additionally, we highlight key discussions held at the workshop. Our main conclusion is that it is unclear what the ideal observable is to make this link between formation scenarios and exoplanet atmospheres, whether it be multiple elemental abundance ratios, measuring refractory budgets, or something else. Based on discussions held throughout the workshop, we provide several key takeaways of what the workshop attendees feel need the most improvement and exploration within each discipline.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.00435-b31b1b.svg)](https://arxiv.org/abs/2506.00435) | **Spectral Hardening Reveals Afterglow Emergence in Long-Duration Fast X-ray Transients: A Case Study of GRB 250404A/EP250404a**  |
|| Y.-H. I. Yin, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-06-03*|
|*Comments*| *20 pages, 7 figures, 6 tables*|
|**Abstract**|            The prompt emission and afterglow phases of gamma-ray bursts (GRBs) have been extensively studied, yet the transition between these two phases remains inadequately characterized due to limited multiwavelength observational coverage. Among the recent growing samples of fast X-ray transients observed by Einstein Probe (EP), a subgroup of gamma-ray bursts are captured with long-duration X-ray emission, potentially containing featured evolution from prompt emission to the afterglow phase. In this Letter, we present a detailed analysis of GRB 250404A/EP250404a, a bright fast X-ray transient detected simultaneously by EP and Fermi/GBM in X-rays and gamma-rays. Its continuous X-ray emission reveals a long-duration tail, accompanied by distinct spectral evolution manifested by the spectral index $\alpha_{\rm X}$ with an initial softening, followed by an evident hardening, eventually reaching a plateau at the value of $\sim$ -2. Early optical and near-infrared observations enable broadband modeling with forward- and reverse-shock components, confirming that the X-ray hardening signals the emergence of the external-shock afterglow. From this spectral hardening we infer that the prompt phase in soft X-rays lasted $\sim300$ s, which is more than three times longer than the gamma-ray $T_{90}$. This well-tracked soft-hard-flat spectral pattern provides a clear indication of afterglow emergence from the fading prompt emission and offers a practical criterion for identifying a distinct population of GRBs among fast X-ray transients, even when the detection of the gamma-ray counterpart or obvious temporal break is absent.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.01632-b31b1b.svg)](https://arxiv.org/abs/2506.01632) | **Probing the Turbulent Corona and Heliosphere Using Radio Spectral Imaging Observation during the Solar Conjunction of Crab Nebula**  |
|| P. Zhang, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-06-03*|
|*Comments*| **|
|**Abstract**|            Measuring plasma parameters in the upper solar corona and inner heliosphere is challenging because of the region's weakly emissive nature and inaccessibility for most in situ observations. Radio imaging of broadened and distorted background astronomical radio sources during solar conjunction can provide unique constraints for the coronal material along the line of sight. In this study, we present radio spectral imaging observations of the Crab Nebula (Tau A) from June 9 to June 22, 2024 when it was near the Sun with a projected heliocentric distance of 5 to 27 solar radii, using the Owens Valley Radio Observatory's Long Wavelength Array (OVRO-LWA) at multiple frequencies in the 30--80 MHz range. The imaging data reveal frequency-dependent broadening and distortion effects caused by anisotropic wave propagation through the turbulent solar corona at different distances. We analyze the brightness, size, and anisotropy of the broadened images. Our results provide detailed observations showing that the eccentricity of the unresolved source increases as the line of sight approaches the Sun, suggesting a higher anisotropic ratio of the plasma turbulence closer to the Sun. In addition, the major axis of the elongated source is consistently oriented in the direction perpendicular to the radial direction, suggesting that the turbulence-induced scattering effect is more pronounced in the direction transverse to the coronal magnetic field. Lastly, when the source undergoes large-scale refraction as the line of sight passes through a streamer, the apparent source exhibits substructures at lower frequencies. This study demonstrates that observations of celestial radio sources with lines of sight near the Sun provide a promising method for measuring turbulence parameters in the inner heliosphere.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.01733-b31b1b.svg)](https://arxiv.org/abs/2506.01733) | **Observation of the Crab Nebula with the Single-Mirror Small-Size Telescope stereoscopic system at low altitude**  |
|| C. Alispach, et al. -- incl., <mark>A. Müller</mark> |
|*Appeared on*| *2025-06-03*|
|*Comments*| *22 pages, 20 figures. Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            The Single-Mirror Small-Size Telescope (SST-1M) stereoscopic system is composed of two Imaging Atmospheric Cherenkov Telescopes (IACTs) designed for optimal performance for gamma-ray astronomy in the multi-TeV energy range. It features a 4-meter-diameter tessellated mirror dish and an innovative SiPM-based camera. Its optical system features a 4-m diameter spherical mirror dish based on the Davies-Cotton design, maintaining a good image quality over a large FoV while minimizing optical aberrations. In 2022, two SST-1M telescopes were installed at the Ondřejov Observatory, Czech Republic, at an altitude of 510 meters above sea level, and have been collecting data for commissioning and astronomical observations since then. We present the first SST-1M observations of the Crab Nebula, conducted between September 2023 and March 2024 in both mono and stereoscopic modes. During this observation period, 46 hours for the SST-1M-1 and 52 hours for the SST-1M-2 were collected for which 33 hours are in stereoscopic mode. We use the Crab Nebula observation to validate the expected performance of the instrument, as evaluated by Monte Carlo simulations carefully tuned to account for instrumental and atmospheric effects. We determined that the energy threshold at the analysis level for the zenith angles below $30^\circ$ is 1 TeV for mono mode and 1.3 TeV for stereo mode. The energy and angular resolutions are approximately 20\% and $0.18^\circ$ for mono mode and 10\% and $0.10^\circ$ for stereo mode, respectively. We present the off-axis performance of the instrument and a detailed study of systematic uncertainties. The results of a full simulation of the telescope and its camera is compared to the data for the first time, allowing a deep understanding of the SST-1M array performance.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.01771-b31b1b.svg)](https://arxiv.org/abs/2506.01771) | **SiO and a super-stellar C/O ratio in the atmosphere of the giant exoplanet WASP-121b**  |
|| T. M. Evans-Soma, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>C. Gapp</mark> |
|*Appeared on*| *2025-06-03*|
|*Comments*| *Published in Nature Astronomy (publisher version is Open Access)*|
|**Abstract**|            Refractory elements such as iron, magnesium, and silicon can be detected in the atmospheres of ultrahot giant planets. This provides an opportunity to quantify the amount of refractory material accreted during formation, along with volatile gases and ices. However, simultaneous detections of refractories and volatiles have proved challenging, as the most prominent spectral features of associated atoms and molecules span a broad wavelength range. Here, using a single JWST observation of the ultrahot giant planet WASP-121b, we report detections of H$_2$O (5.5-13.5$\sigma$), CO (10.8-12.8$\sigma$), and SiO (5.7-6.2$\sigma$) in the planet's dayside atmosphere, and CH$_4$ (3.1-5.1$\sigma$) in the nightside atmosphere. We measure super-stellar values for the atmospheric C/H, O/H, Si/H, and C/O ratios, which point to the joint importance of pebbles and planetesimals in giant planet formation. The CH$_4$-rich nightside composition is also indicative of dynamical processes, such as strong vertical mixing, having a profound influence on the chemistry of ultrahot giant planets.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.00669/./CO_depl.png', 'tmp_2506.00669/./xsection_combined.png', 'tmp_2506.00669/./compared_co_ratios_split_v2.png']
copying  tmp_2506.00669/./CO_depl.png to _build/html/
copying  tmp_2506.00669/./xsection_combined.png to _build/html/
copying  tmp_2506.00669/./compared_co_ratios_split_v2.png to _build/html/
exported in  _build/html/2506.00669.md
    + _build/html/tmp_2506.00669/./CO_depl.png
    + _build/html/tmp_2506.00669/./xsection_combined.png
    + _build/html/tmp_2506.00669/./compared_co_ratios_split_v2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\water}{H_2O}$
$\newcommand{\sotwo}{SO_2}$
$\newcommand{\cotwo}{CO_2}$
$\newcommand{\methane}{CH_4}$
$\newcommand{\ammonia}{NH_3}$</div>



<div id="title">

# On Linking Planet Formation Models, Protoplanetary Disk Properties, and Mature Gas Giant Exoplanet Atmospheres

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.00669-b31b1b.svg)](https://arxiv.org/abs/2506.00669)<mark>Appeared on: 2025-06-03</mark> -  _24 pages (37 pages with references), 8 figures, 1 table. Submitted to AAS Journals. This article is intended to reflect the discussions and perspectives of workshop participants, and not provide a comprehensive review of the fields covered_

</div>
<div id="authors">

Adina~D.~Feinstein, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>I. J. M. Crossfield</mark>, <mark>L. Kreidberg</mark>, <mark>G. Perotti</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** Measuring a single elemental ratio (e.g., carbon-to-oxygen) provides insufficient information for understanding the formation mechanisms and evolution that affect our observations of gas  giant planet atmospheres. Although the fields of planet formation, protoplanetary disks, and exoplanets are well established and interconnected, our understanding of how to self-consistently and accurately link the theoretical and observational aspects of these fields together is lacking. To foster interdisciplinary conversations, the Max-Planck Institut für Astronomie (MPIA) hosted a week-long workshop called, "Challenge Accepted: Linking Planet Formation with Present-Day Atmospheres."  Here, we summarize the latest theories and results in planet formation modeling, protoplanetary disk observations, and atmospheric observations of gas giant atmospheres to address one of the challenges of hosting interdisciplinary conferences: ensuring everyone is aware of the state-of-the-art results and technical language from each discipline represented. Additionally, we highlight key discussions held at the workshop. Our main conclusion is that it is unclear what the ideal observable is to make this link between formation scenarios and exoplanet atmospheres, whether it be multiple elemental abundance ratios, measuring refractory budgets, or something else. Based on discussions held throughout the workshop, we provide several key takeaways of what the workshop attendees feel need the most improvement and exploration within each discipline.

</div>

<div id="div_fig1">

<img src="tmp_2506.00669/./CO_depl.png" alt="Fig1" width="100%"/>

**Figure 1. -** The impact of CO depletion on the composition of hot Jupiters using the formation model of [Penzlin, Booth and Kirk (2024)](). Here, 90\% of the CO in the fiducial model has been replaced with either $C_2$$H_6$, $CO_2$, or $CH_3$OH. In all of the different disk models, the composition of hot Jupiters that underwent high-eccentricity migration differs from those that did not. However, different assumptions about CO depletion in the disk produce larger changes in the C/O ratio than the differences in formation history. (*fig:co_comp*)

</div>
<div id="div_fig2">

<img src="tmp_2506.00669/./xsection_combined.png" alt="Fig4" width="100%"/>

**Figure 4. -** The cross-sections for molecular species which could exist in a (a) hot Jupiter at 1000 K and (b) ultra hot Jupiter at 2500 K atmosphere. The wavelengths plotted here are within the range of JWST's NIRISS, NIRCam, NIRSpec, and MIRI/LRS. With access to the broader wavelength coverage of JWST, we are now able to potentially detect multiple absorption features from the same species and multiple species, simultaneously. Cross sections computed following the methods and sources of opacity listed in [Welbanks and Madhusudhan (2021)](). We note that several versions of this plot already exist in the literature \citep[e.g.,][]{madhusudhan19}. (*fig:spaghetti*)

</div>
<div id="div_fig3">

<img src="tmp_2506.00669/./compared_co_ratios_split_v2.png" alt="Fig5" width="100%"/>

**Figure 5. -** A comparison of the measured carbon-to-oxygen (C/O) ratio for planets, as observed with different instruments/wavelength coverages (see Table \ref{tab:exoplanet_c2o} for details). Panels marked with $\triangle$ highlight upper limits on the C/O. Grayed out squares do not have C/O measurements for those planets with those instruments. While the majority of measurements of C/O for each of these planets are consistent within $2\sigma$, the difference in wavelength coverage, and thus which molecular species are detected, change the measured C/O for the same planet.  (*fig:exoplanet_c2o*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.00669"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

507  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
